# SENG 550 Final Project
### Use the Amazon Appliances reviews dataset to develop a classifier or sentiment analyzer that can predict whether a given review is favorable or not

## Abstract

Our project uses the Amazon Appliances [reviews dataset](https://amazon-reviews-2023.github.io/) to develop a sentiment analyzer classification model. By combining star ratings and textual content, the model is trained to predict whether a review leans positively or negatively towards a product. This approach offers a way to quickly grasp the general sentiment of a product and assist shoppers in filtering through large volumes of product feedback more efficiently.

## Introduction

### Selected Problem

The problem aims to distinguish between favourable and unfavourable appliance reviews based on their text and accompanying reviews.

### Why is it Important?

Spending time reading review after review on a product becomes a burden. It is easy to misinterpret the mood behind a set of comments online which can easily lead to poor purchase decisions. A quick, automated sentiment indicator can ease the burden and establish a neutral decision making process.

### What have Others Done in this Space?

Researchers have performed [sentiment analysis](https://medium.com/@nafisaidris413/a-beginners-guide-for-product-review-sentiment-analysis-0de1f451167d) using Machine Learning and Natural Language Processing to automatically classify reviews as positive, negative, or neutral. Not only has sentiment analysis been applied to product reviews, it has also been applied to [social media](https://buffer.com/social-media-terms/sentiment-analysis) to determine how people perceive and talk about products and brands. This proves that data-driven classifiers are able to provide sentiment analysis scores for assist people in their daily lives, whether its to determine how their personal brand is viewed or how a to make an informed purchase through product review.

### Existing gaps?

Current solutions rely on product ratings to provide consumers with a sense of trust and quality to help them make purchasing decisions. This can be seen simply by going to any Amazon product and checking the reviews. Some reviews are informative with many positives about the product, though the product receives less than a rating of 5-stars, or the review is not informative whatsoever with a rating of 5-stars. Other times the reviews are clearly biased or the customer who leaves the review is disgruntled, leading to a 1- or 2-star rating. Using a combination of product star rating and textual review content, we are attempting to reveal patterns in product reviews that a rating alone might miss.

### Data Analysis Questions

1. Does text-based features add value beyond just a numerical rating?
2. Are there certain words which portray a stronger positve or negative sentiment?
3. How will adding text preprocessing impact accuracy?
4. Which models work best with this data?

### What is Proposed

We are proposing a text classification pipeline that merges a product's star rating and textual features.

### What are your Main Findings?

To determine customer opinion on various products within the Appliance category in Amazon's online store.

## Methodology

### Exploration of Data Features and Refinement of Feature Space

In this section, we focused on understanding the raw data collected from the collected [datasets](https://amazon-reviews-2023.github.io/) and transform them into a format suitable for model training. We begin by loading the Amazon Appliance reviews dataset and its corresponding metadata. We will explore the structure of the data, examine the distribution of fields we are interested in (like ratings), and assess the overall quality of the text reviews associated with the products. After we gain a thorough understanding, we apply a series of preprocessing techniques to clean and refine the text data. The goal here is to ultimately develop a set of features that can be fed into a machine learning model for sentiment classification.

#### Key Steps

1. **Loading the Data:**
We will loaf the `Appliances.jsonl` (reviews) and `meta_Appliances.jsonl` (metadata) using Apache Spark to avoid memory overload

2. **Initial Inspection and Basic Statistics:**
We will look at a few sample rows, check data types, count missing values, and examine distributions.

3. **Textual Data Exploration:**
We consider the nature of each review such as its length, the character composition, and common words. This should help guide our text cleaning decision.

4. **Data Cleaning:**
We clean the text by methods such as lowercasing the characters, removing punctuation, stripping leading and/or trailing whitespaces.

5. **Feature Transformation:**
We will use Spark Machine Learning's feature extraction tools to convert raw text into numeric features that are typically suitable for machine learning models.

#### Load the Data

The datasets are provided in `*.jsonl` format, which means each line is a separate JSON object representing a single review or product's metadata. We will use `SparkSession` to read the files which will handle the data in a distributed manner, effectively avoiding a potential kernel crash. Spark's lazy evaluation, transformations, and actions will manage memory usage of the large `.jsonl` files

The two main data sources:
1. **Review File (`Appliances.jsonl`):**
Contains user-level reviews with fields such as `rating`, `title`, `text`, `verified_purchase`, `helpful_vote`, and `timestamp`.

2. **Metada File (`meta_Appliaces.jsonl`):**
Contains product-level information like `main_category`, `average_rating`, `description`, `features`, and `price`.

In [ ]:
import os
from pyspark.sql import SparkSession, types
from pyspark.sql import functions
from pyspark.ml import feature
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType, ArrayType, MapType

# Start a Spark Session
spark = (
    SparkSession.builder
        .master("local")
        .appName("Amazon Review Analysis") 
        .config("spark.executor.memory", "4g")
        .config("spark.driver.memory", "4g")
        .getOrCreate()
)

# Only using columns needed for analysis for Reviews
reviews_schema = StructType([
    StructField("rating", FloatType(), True),
    StructField("title", StringType(), True),
    StructField("text", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("parent_asin", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("verified_purchase", BooleanType() , True),
    StructField("helpful_vite", IntegerType() , True)
])

# Only using columns needed for analysis for Metadata
meta_schema = StructType([
    StructField("main_category", StringType(), True),
    StructField("title", StringType(), True),
    StructField("average_rating", FloatType(), True),
    StructField("rating_number", IntegerType(), True),
    StructField("features", ArrayType(StringType(), True), True),
    StructField("description", ArrayType(StringType(), True), True),
    StructField("price", FloatType(), True),
    StructField("store", StringType(), True),
    StructField("categories", ArrayType(StringType(), True), True),
    StructField("details", MapType(StringType(), StringType(), True), True),
    StructField("parent_asin", StringType(), True)
])

# Point to the location where the .jsonl files are
reviews_file = "Appliances.jsonl"
meta_file = "meta_Appliances.jsonl"

# Use the schema when reading the JSON file for Reviews
df_reviews = spark.read.schema(reviews_schema).json(reviews_file)

# Use the schema when reading the JSON file for Meta
df_meta = spark.read.schema(meta_schema).json(meta_file)

# Show the loaded DataFrame to ensure successful creation
df_reviews.show(5, truncate=False)
df_meta.show(5, truncate=False)


+------+-----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+-----------+----------------------------+-----------------+------------+
|rating|title            |text                                                                                                                                                                                                                                                                                                       |asin      |parent_asin|user_id                     |verified_purchase|helpful_vite|
+------+-----------------+----------------------------------------------------------------------------------------------------------------------------------------------------------

The schemas printed above gives an overview of the fields that are being used in the model for both the Appliances Reviews and Appliances Metadata. Spark will naturally run on **PORT 4040**.

#### Initial Inspection & Basic Statistics

First it is important to understand the size of the dataset we are dealing with and the distribution of ratings. Using Spark actions like `show()` and `count()` we determine some initial statistics about both datasets which will be helpful to visualize them.

In [21]:
print(f"From the Reviews dataset")
print(f"Number of Reviews: ", df_reviews.count())
print(f"Number of distinct products: ", df_reviews.select("asin").distinct().count())

print(f"From the Meta dataset")
print(f"Amount of Metadata", df_meta.count())

From the Reviews dataset


Number of Reviews:  2128605


Number of distinct products:  104237
From the Reviews dataset


Amount of Metadata 94327


In [14]:
spark.stop()